# Fetching LA Metro Data Using AWS
## Based On UCLA ITS Data Camp, Day 1 Exercise *demo added at end*
## Retrieving Data via API Calls

### Exercise 2: Getting data from LA Metro
Most of the time that you are querying APIs for data it will not be so straightforward to get it into a tidy format. Instead, what you will usually want to do is inspect the response content first before deciding how to proceed. Let's take a look at data from [LA Metro's Developer Portal](https://developer.metro.net/). Going to the [Metro Bus & Rail Real-time Arrivals](https://developer.metro.net/portfolio-item/real-time-arrivals/) page, we can see a variety of APIs that are publicly available. Take a look at all the [feeds](https://developer.metro.net/introduction/realtime-api-overview/realtime-api-returning-json/) returning JSON-formatted content, including route information, stop information, and realtime vehicle location information.

You will notice that instead of GeoJSON, it is in a slightly different format that requires just a bit of wrangling to get it in the right format.

##### Create an API Call
Pick any of the Metro routes and, following the structure in the example, make a call to get all the current vehicles on that route. Once we get the response (assuming it is successful), let's take a look at the content.

In [4]:
import requests
import pandas as pd

# TODO: Write the statement to call the Metro API and get all vehicles for a particular route
#       and store the response as resp
# (No need to import the requests package again)
resp = requests.get('http://api.metro.net/agencies/lametro/routes/733/vehicles/')
print(resp.status_code)
# TODO: Store the JSON content as `data`
data = resp.json()

200


In [5]:
data

{'items': [{'run_id': '733_96_0',
   'seconds_since_report': 9,
   'heading': 95.0,
   'route_id': '733',
   'predictable': True,
   'latitude': 34.043513,
   'longitude': -118.3178792,
   'id': '9532'},
  {'run_id': '733_100_1',
   'seconds_since_report': 87,
   'heading': 240.0,
   'route_id': '733',
   'predictable': True,
   'latitude': 34.04401,
   'longitude': -118.299858,
   'id': '9374'},
  {'run_id': '733_97_0',
   'seconds_since_report': 169,
   'heading': 36.0,
   'route_id': '733',
   'predictable': True,
   'latitude': 34.036171,
   'longitude': -118.26049,
   'id': '9295'},
  {'run_id': '733_97_0',
   'seconds_since_report': 31,
   'heading': 135.0,
   'route_id': '733',
   'predictable': True,
   'latitude': 34.014172,
   'longitude': -118.496391,
   'id': '9309'},
  {'seconds_since_report': 31,
   'heading': -4.0,
   'route_id': '733',
   'predictable': False,
   'latitude': 34.056992,
   'longitude': -118.231216,
   'id': '9384'},
  {'run_id': '733_100_1',
   'seconds_

You will notice that instead of GeoJSON, it is in a slightly different format. We can convert a list of key, value pairs into a Pandas dataframe easily by `df = pd.DataFrame(dict)`. Let's go ahead and convert the json output into a dict. _Hint: Make sure you access the list part of the JSON output!_

In [6]:
# TODO: Convert the JSON output to a dataframe
metro_df = pd.DataFrame(data['items'])

# Examine the head of the dataframe
metro_df.head()

,run_id,seconds_since_report,heading,route_id,predictable,latitude,longitude,id
0,733_96_0,9,95.0,733,True,34.043513,-118.317879,9532
1,733_100_1,87,240.0,733,True,34.044010,-118.299858,9374
2,733_97_0,169,36.0,733,True,34.036171,-118.260490,9295
3,733_97_0,31,135.0,733,True,34.014172,-118.496391,9309
4,NaN,31,-4.0,733,False,34.056992,-118.231216,9384


##### Add a Column to the DataFrame
One thing you will notice is that when we made the dataframe above, we are missing the timestamp of the query. If we plan to write out the data for analysis later, we need to add the time of the query as a column value. The easiest way to get the current time in Python is through the [datetime](https://docs.python.org/2/library/datetime.html) package. Take a little bit of time to look through the documentation with a particular focus on the `now()` method.

Once we get the value of the current time, we can add it as a new column value to our current dataframe. Create an additonal column `call_time`. In the function, get the current timestamp of the call and add it as the value for that column.

In [7]:
# Import the datetime module
from datetime import datetime

# TODO: Get the current time
now = datetime.now()

# TODO: Add the current time as a value to the dataframe column `call_time`
metro_df['call_time'] = now

In [8]:
metro_df.head()

,run_id,seconds_since_report,heading,route_id,predictable,latitude,longitude,id,call_time
0,733_96_0,9,95.0,733,True,34.043513,-118.317879,9532,2019-10-13 21:43:33.886955
1,733_100_1,87,240.0,733,True,34.044010,-118.299858,9374,2019-10-13 21:43:33.886955
2,733_97_0,169,36.0,733,True,34.036171,-118.260490,9295,2019-10-13 21:43:33.886955
3,733_97_0,31,135.0,733,True,34.014172,-118.496391,9309,2019-10-13 21:43:33.886955
4,NaN,31,-4.0,733,False,34.056992,-118.231216,9384,2019-10-13 21:43:33.886955


##### Wrap the API Call in a Function
Let's create a function to take a Route ID and make the API call for all realtime vehicle locations on that route. Add in the code we used in the block above to also create a column with the time we called the API.

_Function Input:_ Route ID  
_Function Output:_ Response Dataframe with the content response 

In [9]:
# TODO: Create the function
def get_vehicles_byroute(routenum):
    resp = requests.get('http://api.metro.net/agencies/lametro/routes/%s/vehicles/' % routenum)
    print(resp.status_code)
    # TODO: Store the JSON content as `data`
    data = resp.json()
    # TODO: Convert the JSON output to a dataframe
    routedata = pd.DataFrame(data['items'])

    # Examine the head of the dataframe
    #routedata.head()
    # Import the datetime module
    from datetime import datetime

    # TODO: Get the current time
    now = datetime.now()

    # TODO: Add the current time as a value to the dataframe column `call_time`
    routedata['call_time'] = now
    return routedata

Let's take a look to make sure our function is working correctly. Run the cell below to confirm that you are getting the desired result. Go ahead and try changing the input and see how the output changes.

In [10]:
# Call the function for one of the routes
routedata = get_vehicles_byroute(720)

# Examine the head of the dataframe
routedata.head()

200


,id,route_id,predictable,run_id,latitude,longitude,heading,seconds_since_report,call_time
0,9363,720,True,720_1144_1,34.023880,-118.173531,270.0,26,2019-10-13 21:43:41.812069
1,9360,720,True,720_1144_1,34.040653,-118.239937,360.0,106,2019-10-13 21:43:41.812069
2,9560,720,True,720_1144_1,34.062626,-118.348305,270.0,133,2019-10-13 21:43:41.812069
3,9366,720,True,720_1122_0,34.028198,-118.487015,45.0,79,2019-10-13 21:43:41.812069
4,9487,720,True,720_1144_1,34.015876,-118.487312,136.0,28,2019-10-13 21:43:41.812069


##### Add Functionality
Great! Now we are able to change the route number and get a dataframe with the current location of all vehicles on the route. One of the next things we might want to do would be to get data from the route throughout the day and store it for later analysis. To do that we are going to need to add the following functionality into our function:

1. Write out the csv to the a file in our `data/processed` folder. Let's set the filename to the format `lametro_[routenum]_[timestamp].csv` (Eg. `lametro_720_2019-09-10-22-26-52.csv`). To write out the file, go ahead and use [Panda's method](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.to_csv.html) for writing out a csv file.
2. Add conditional logic to only write out the file if the call was a success. If the call was not successful, print out the error message. Take a look [here](https://2.python-requests.org/en/master/user/quickstart/#response-status-codes) for some guidance.

As the function gets a bit more complex, please add appropriate code comments inside to quickly convey the purpose of each code block.


In [11]:
print(datetime.now())

2019-10-13 21:43:46.665977


In [12]:
# TODO: Re-write the function with the requested features
def get_vehicles_byroute(routenum):
    #make request with requested route number
    resp = requests.get('http://api.metro.net/agencies/lametro/routes/%s/vehicles/' % routenum)
    #print(resp.status_code)
    
    #check if call is successful
    if resp.status_code != requests.codes.ok:
        print('API call unsuccessful')
        resp.raise_for_status()
        return
        
    # TODO: Store the JSON content as `data`
    data = resp.json()
    # TODO: Convert the JSON output to a dataframe
    routedata = pd.DataFrame(data['items'])

    # Examine the head of the dataframe
    #routedata.head()
    # Import the datetime module
    from datetime import datetime

    # TODO: Get the current time
    now = datetime.now()

    # TODO: Add the current time as a value to the dataframe column `call_time`
    routedata['call_time'] = now
    
    #write to CSV
    #routedata.to_csv('data/processed/lametro_%s_%s.csv' % (routenum, now))
    fname = f"lametro_{routenum}_{now.strftime('%Y-%m-%d-%H-%M-%S')}.csv"
    fpath = 'data/processed/' + fname
    routedata.to_csv(fpath, index=False)
    return routedata

Let's go ahead and call the function again for one of the routes to ensure that it was written out correctly.

In [13]:
get_vehicles_byroute(704)

,run_id,seconds_since_report,heading,route_id,predictable,latitude,longitude,id,call_time
0,704_168_1,9,270.0,704,True,34.090837,-118.328855,9302,2019-10-13 21:43:50.215339
1,704_168_1,9,230.0,704,True,34.071878,-118.403512,9524,2019-10-13 21:43:50.215339
2,704_152_0,9,72.0,704,True,34.041370,-118.461365,9545,2019-10-13 21:43:50.215339
3,704_152_0,9,92.0,704,True,34.090603,-118.372377,9586,2019-10-13 21:43:50.215339
4,704_152_0,36,134.0,704,True,34.078011,-118.266266,9292,2019-10-13 21:43:50.215339
5,704_168_1,7,278.0,704,True,34.057320,-118.230659,9284,2019-10-13 21:43:50.215339
6,704_152_0,7,220.0,704,True,34.015850,-118.500000,9301,2019-10-13 21:43:50.215339
7,704_152_0,34,154.0,704,True,34.082951,-118.273636,9344,2019-10-13 21:43:50.215339


Check your data folder - if everything was successful, you should see a CSV file with the data from the call.
##### Introduction to variable-length arguments 
We've now built a function that, for a given route, will get current vehicle location data, format it into a dataframe, and write it out to a CSV file with the current datetime. What if we were interested in 2 routes? or 3 routes? Let's build another function that takes as input a _variable number of route numbers_ and then gets the vehicle data for each of them.
  
We will do this through the [_*args_ syntax](https://www.geeksforgeeks.org/args-kwargs-python/). Following that syntax, create a function called `get_vehicles_byroutes` that takes in a variable number of route numbers. For each route number, the function should call our other function `get_vehicles_byroute`. Between each call to our original function, add a 5 second pause to reduce the load on the server.

In [14]:
import time

# TODO: Finish composing the function
def get_vehicles_byroutes(*routes):
    for route in routes:
        get_vehicles_byroute(route)
        time.sleep(5)

##### Create a Loop to run the Function 
Great! We now have a function that calls the Metro's API, records the location of all vehicles for a particular route(s), logs the current timestamp, and saves the file in a location of our choosing. Let's (1) pick a few routes we want to get data from and (2) create a loop that runs the `get_vehicles_byroutes` function 1x per minute, for 5 minutes with those route numbers as the input.


In [20]:
# TODO: Execute the function 5x, each time separated by a minute
for _ in range(5):
    get_vehicles_byroutes(20, 720, 33)
    time.sleep(60)

In [ ]:
#### Fetch a day's worth of LA Metro data

Run this cell 